In [1]:
%load_ext bigdata
%hive_start
%timeout 300

In [2]:
%%hive
DROP TABLE IF EXISTS t0;
CREATE TABLE t0 (
    c1 STRING,
    c2 ARRAY<CHAR(1)>, 
    c3 MAP<STRING, INT>
    )
    ROW FORMAT DELIMITED 
        FIELDS TERMINATED BY '\t'
        COLLECTION ITEMS TERMINATED BY ','
        MAP KEYS TERMINATED BY '#'
        LINES TERMINATED BY '\n';
LOAD DATA LOCAL INPATH 'data.tsv' INTO TABLE t0;

DROP TABLE IF EXISTS t0;
OK
Time taken: 16.564 seconds
CREATE TABLE t0 (
    c1 STRING,
    c2 ARRAY<CHAR(1)>, 
    c3 MAP<STRING, INT>
    )
    ROW FORMAT DELIMITED 
        FIELDS TERMINATED BY '\t'
        COLLECTION ITEMS TERMINATED BY ','
        MAP KEYS TERMINATED BY '#'
        LINES TERMINATED BY '\n';
OK
Time taken: 1.327 seconds
LOAD DATA LOCAL INPATH 'data.tsv' INTO TABLE t0;
Loading data to table default.t0
OK
Time taken: 2.485 seconds


In [3]:
%%hive 
SELECT * FROM t0;

SELECT * FROM t0;
OK
E	["b","g","f"]	{"jjj":3,"bbb":0,"ddd":9,"ggg":8,"hhh":2}
A	["a","f","c"]	{"ccc":2,"ddd":0,"aaa":3,"hhh":9}
B	["f","e","a","c"]	{"ddd":2,"ggg":5,"ccc":6,"jjj":1}
A	["a","b"]	{"hhh":9,"iii":5,"eee":7,"bbb":1}
C	["f","g","d","a"]	{"iii":6,"ddd":5,"eee":4,"jjj":3}
A	["c","d"]	{"bbb":2,"hhh":0,"ccc":4,"fff":1,"aaa":7}
A	["g","d","a"]	{"aaa":5,"fff":8,"ddd":2,"iii":0,"jjj":7,"ccc":1}
B	["b","a"]	{"fff":3,"hhh":1,"ddd":2}
E	["d","e","a","f"]	{"eee":4,"ccc":5,"iii":9,"fff":7,"ggg":6,"bbb":0}
B	["d","b","g","f"]	{"bbb":7,"jjj":9,"fff":5,"iii":4,"ggg":2,"eee":3}
C	["d","c","f","b"]	{"hhh":6,"eee":4,"iii":0,"fff":2,"jjj":1}
C	["d","e","a","c"]	{"bbb":7,"iii":6,"ggg":9}
D	["g","e","f","b"]	{"bbb":9,"aaa":3,"ccc":6,"fff":4,"eee":2}
E	["c","f"]	{"aaa":8,"ddd":5,"jjj":1}
B	["d","b"]	{"ccc":0,"jjj":6,"fff":7,"ddd":3,"aaa":2}
D	["f","e"]	{"ccc":0,"eee":6,"bbb":9,"ddd":3}
E	["e","b","f"]	{"bbb":6,"iii":3,"hhh":5,"fff":4,"ggg":9,"ddd":2}
D	["g","a"]	{"hhh":4,"jjj":5,"ccc":9}
E	["e",

In [10]:
%%hive
DROP TABLE IF EXISTS dataTable; 
CREATE TABLE dataTable (clave STRING, valor INT);
INSERT OVERWRITE TABLE dataTable 
SELECT explode(c3) FROM t0;

DROP TABLE IF EXISTS dataTable; 
OK
Time taken: 0.407 seconds
CREATE TABLE dataTable (clave STRING, valor INT);
OK
Time taken: 0.048 seconds
INSERT OVERWRITE TABLE dataTable 
SELECT explode(c3) FROM t0;
Query ID = root_20200228152959_0edd50e2-50ca-4be4-8cb4-b2c5277af9c9
Total jobs = 3
Launching Job 1 out of 3
Number of reduce tasks is set to 0 since there's no reduce operator
Starting Job = job_1582902829190_0002, Tracking URL = http://5c70801abc21:8088/proxy/application_1582902829190_0002/
Kill Command = /usr/local/hadoop/bin/hadoop job  -kill job_1582902829190_0002
Hadoop job information for Stage-1: number of mappers: 1; number of reducers: 0
2020-02-28 15:30:08,427 Stage-1 map = 0%,  reduce = 0%
2020-02-28 15:30:14,034 Stage-1 map = 100%,  reduce = 0%, Cumulative CPU 2.51 sec
MapReduce Total cumulative CPU time: 2 seconds 510 msec
Ended Job = job_1582902829190_0002
Stage-4 is selected by condition resolver.
Stage-3 is filtered out by condition resolver.
Stage-5 is filtered out by c

In [14]:
%%hive
SELECT clave FROM dataTable LIMIT 10;

SELECT clave FROM dataTable LIMIT 10;
OK
jjj
bbb
ddd
ggg
hhh
ccc
ddd
aaa
hhh
ddd
Time taken: 0.143 seconds, Fetched: 10 row(s)


In [15]:
%%hive
DROP TABLE IF EXISTS word_counts;
CREATE TABLE word_counts
AS
    SELECT word,count(1) AS count
    FROM
        (SELECT clave AS word FROM dataTable) w
GROUP BY
    word
ORDER BY
    word;

DROP TABLE IF EXISTS word_counts;
OK
Time taken: 0.006 seconds
CREATE TABLE word_counts
AS
    SELECT word,count(1) AS count
    FROM
        (SELECT clave AS word FROM dataTable) w
GROUP BY
    word
ORDER BY
    word;
Query ID = root_20200228153130_9792265e-5cb5-4d4c-a420-e3879729959f
Total jobs = 2
Launching Job 1 out of 2
Number of reduce tasks not specified. Estimated from input data size: 1
In order to change the average load for a reducer (in bytes):
  set hive.exec.reducers.bytes.per.reducer=<number>
In order to limit the maximum number of reducers:
  set hive.exec.reducers.max=<number>
In order to set a constant number of reducers:
  set mapreduce.job.reduces=<number>
Starting Job = job_1582902829190_0003, Tracking URL = http://5c70801abc21:8088/proxy/application_1582902829190_0003/
Kill Command = /usr/local/hadoop/bin/hadoop job  -kill job_1582902829190_0003
Hadoop job information for Stage-1: number of mappers: 1; number of reducers: 1
2020-02-28 15:31:39,000 Stage-1 map = 0%

In [16]:
%%hive
SELECT * FROM word_counts;

SELECT * FROM word_counts;
OK
aaa	13
bbb	16
ccc	23
ddd	23
eee	15
fff	20
ggg	13
hhh	16
iii	18
jjj	18
Time taken: 0.175 seconds, Fetched: 10 row(s)


In [17]:
%hive_quit

In [18]:
## Copia Archivos a sistema HDFS
##
## Se usan un directorio temporal en el HDFS. La siguiente
## instrucción muestra el contenido del dicho directorio
##
!hdfs dfs -ls /tmp

Found 2 items
drwxrwx---   - root supergroup          0 2020-02-28 15:14 /tmp/hadoop-yarn
drwxrwxrwx   - root supergroup          0 2020-02-28 15:19 /tmp/hive


In [19]:
## Crea la carpeta wordcount en el hdfs
##
!hdfs dfs -mkdir /tmp/wordcount

In [20]:
## Verifica la creación de la carpeta
##
!hdfs dfs -ls /tmp/

Found 3 items
drwxrwx---   - root supergroup          0 2020-02-28 15:14 /tmp/hadoop-yarn
drwxrwxrwx   - root supergroup          0 2020-02-28 15:19 /tmp/hive
drwxr-xr-x   - root supergroup          0 2020-02-28 15:33 /tmp/wordcount


In [21]:
## Copia los archvios del directorio local wordcount/
## al directorio /tmp/wordcount/ en el hdfs
##
!hdfs dfs -copyFromLocal 'data.tsv'  /tmp/wordcount/

In [22]:
##
## Verifica que los archivos esten copiados
## en el hdfs
##
!hdfs dfs -ls /tmp/wordcount

Found 1 items
-rw-r--r--   1 root supergroup       1394 2020-02-28 15:33 /tmp/wordcount/data.tsv


In [49]:
####################################################################################################################
## Generación del script y ajuste del código

In [23]:
%%writefile wordcount.hql

DROP TABLE IF EXISTS t0;
CREATE TABLE t0 (
    c1 STRING,
    c2 ARRAY<CHAR(1)>, 
    c3 MAP<STRING, INT>
    )
    ROW FORMAT DELIMITED 
        FIELDS TERMINATED BY '\t'
        COLLECTION ITEMS TERMINATED BY ','
        MAP KEYS TERMINATED BY '#'
        LINES TERMINATED BY '\n';
LOAD DATA LOCAL INPATH 'data.tsv' INTO TABLE t0;




DROP TABLE IF EXISTS dataTable; 
CREATE TABLE dataTable (clave STRING, valor INT);
INSERT OVERWRITE TABLE dataTable 
SELECT explode(c3) FROM t0;

DROP TABLE IF EXISTS word_counts;
CREATE TABLE word_counts
AS
    SELECT word,count(1) AS count
    FROM
        (SELECT clave AS word FROM dataTable) w
GROUP BY
    word
ORDER BY
    word;


INSERT OVERWRITE DIRECTORY '/tmp/output'
ROW FORMAT DELIMITED FIELDS TERMINATED BY ','

SELECT * FROM word_counts;

Writing wordcount.hql


In [24]:
## Ejecucion
!hive -S -e 'source wordcount.hql'

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/usr/local/hive/lib/log4j-slf4j-impl-2.6.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/usr/local/hadoop/share/hadoop/common/lib/slf4j-log4j12-1.7.10.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]


In [25]:
### Copia de los resultados a la maquina local (vagrant)
!hadoop fs -copyToLocal /tmp/output output
!ls output/*

output/000000_0


In [26]:
!cat output/000000_0 | head

aaa,13
bbb,16
ccc,23
ddd,23
eee,15
fff,20
ggg,13
hhh,16
iii,18
jjj,18
